In [1]:
import numpy as np
import os
import cv2

#import pandas as pd
#import matplotlib.pyplot as plt
#from PIL import Image
#from PIL import ImageFile

from tqdm import tqdm 

In [2]:
import torch
#import torch.nn as nn
#from torch.nn import functional as F
from torch.utils.data import DataLoader
#from torchvision import models

import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [3]:
from pytorch_grad_cam import GradCAM#, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad

#from pytorch_grad_cam.utils.image import show_cam_on_image


D:\Users\Alex\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from functions_crop import(import_train_folder_dataset,
                      Data,
                      import_test_folder_dataset,
                      Data_test
                     )

# Load Model

In [5]:
                            
#torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8

BBOX_THRESHOLD = 0.75 # From 0-1

PATH = "D:\\Personal\\Alex\\Uni\\MS IA\\2 Curso\\Fall 2022\Artificial Intelligence\\Projects\\Project 1\\Models Save\\GoogleNet_CAM-Plus-ImgClass_v2\\"
save_cropedImg_path = "D:\\Personal\\Alex\\Uni\\MS IA\\2 Curso\\Fall 2022\Artificial Intelligence\\Projects\\Project 1\\data\\Croped_v2\\"

cuda


In [6]:
# Load model
model_save = "weights_googlenet_0.7763157894736842_best.pt"
model_bbox = torch.load(PATH+model_save, map_location=device).to(device)


# Define the Target Layers, for CAM
#target_layers = [model_bbox.resnet.layer4[-1]]
target_layers = [model_bbox.inception5b.branch4]

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model_bbox, target_layers=target_layers, use_cuda=torch.cuda.is_available())



model_bbox.eval()

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

# Load Images

In [7]:
# Path of the files
train_data_path = "D:/Personal/Alex/Uni/MS IA/2 Curso/Fall 2022/Artificial Intelligence/Projects/Project 1/data/Train"
test_data_path = "D:/Personal/Alex/Uni/MS IA/2 Curso/Fall 2022/Artificial Intelligence/Projects/Project 1/data/Test"

In [8]:

img_transform = transforms.Compose([transforms.ToPILImage(),
                                        transforms.Resize((400, 400)),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

In [9]:
# Import the data
train_data, train_labels, train_names = import_train_folder_dataset(train_data_path)
print("Train Dim of Data: ", train_data.shape)
print("Train Dim of Lables: ", train_labels.shape)
print("Train Dim of Lables: ", train_names.shape)

Train Dim of Data:  (1149,)
Train Dim of Lables:  (1149,)
Train Dim of Lables:  (1149,)


C:\Users\Alex\Jupyter NoteBook Master\IA Master\Fall 2022\Artificial Intelligence\Final Project\GoogleNet_CAM-Plus-ImgClass_v2\functions_crop.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return( np.array(img_data_files),np.array(label_data_files),np.array(img_names_files) )


In [10]:
# Import the data
test_data, test_names = import_test_folder_dataset(test_data_path)
print("Test Dim of Data: ", test_data.shape)
print("Test Dim of Lables: ", test_names.shape)

Test Dim of Data:  (380,)
Test Dim of Lables:  (380,)


C:\Users\Alex\Jupyter NoteBook Master\IA Master\Fall 2022\Artificial Intelligence\Final Project\GoogleNet_CAM-Plus-ImgClass_v2\functions_crop.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return( (np.array(img_data_files),np.array(img_data_names)) )


In [11]:
# Create the dataset object
train_Dataset = Data(train_data, train_labels, train_names, img_transform)

print("Train len: ",len(train_Dataset))

# Create the dataset object
test_Dataset = Data_test(test_data, test_names, img_transform) # img_transform_2 img_transform

print("Test len: ",len(test_Dataset))

Train len:  1149
Test len:  380


In [12]:
# Create the dataloaders
train_loader = DataLoader(
    train_Dataset, 
    batch_size=TRAIN_BATCH_SIZE,             
    shuffle=False)

# Create the dataloaders
test_loader = DataLoader(
    test_Dataset, 
    batch_size=TEST_BATCH_SIZE,             
    shuffle=False)

# Crop Images

## Functions

In [13]:
def get_heatmap(image, device, cam_function):
    # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
    grayscale_cam = cam_function(input_tensor=image.to(device), targets=None)

    return grayscale_cam

In [14]:
def bbox(heat_img, threshold, dim = (224,224)):
    H, W = heat_img.shape
    # Create Copy
    heat_2 = np.copy(heat_img)
    # Apply thershold
    heat_2[ heat_2 > threshold ] = 255
    heat_2[ heat_2 != 255 ] = 0
    
    up_bound_coord = -1
    down_bound_coord = -1
    for i in range(len(heat_2)):
        # Uper bound
        up_row = heat_2[i]
        if(up_bound_coord==-1 and 255 in up_row):
            # check if there is image in this row
            up_bound_coord = i
            
        # Lower bound
        low_row = heat_2[-(i+1)]
        if(down_bound_coord==-1 and 255 in low_row):
            # check if there is image in this row
            down_bound_coord = len(heat_2) - (i)
        
    left_bound_coord = -1
    right_bound_coord = -1
    for i in range(len(heat_2[0])):
        # left bound
        left_row = heat_2[:,i]
        if(left_bound_coord==-1 and 255 in left_row):
            # check if there is image in this row
            left_bound_coord = i
            
        # right bound
        right_row = heat_2[:,-(i+1)]
        if(right_bound_coord==-1 and 255 in right_row):
            # check if there is image in this row
            right_bound_coord = len(heat_2) - (i)
    
    return np.array([left_bound_coord*(dim[0]/H), up_bound_coord*(dim[1]/W), right_bound_coord*(dim[0]/H), down_bound_coord*(dim[1]/W)])



# Given a set of images, return the croped versions, based on the cams
def get_cropped_img_batch_base_on_CAM(batch, device, cam_function, threshold = 0.75):
    
    size = (batch.shape[2], batch.shape[3])
    
    
    heat_list = get_heatmap(batch, device, cam_function)
    
    outx = []
    outbox = []
    for img, heat in zip(batch, heat_list):
        box = bbox(heat, threshold = threshold, dim = size)
        
        # Crop the Img
        img_crop = img[:,int(box[1]):int(box[3]), int(box[0]):int(box[2])]
        
        # Resize cropped img to original size
        rr = transforms.Compose([transforms.Resize(size)])
        img_resize = rr(img_crop)
        
        outx.append(img_resize)
        outbox.append(box)
    outx2 = torch.stack(outx, 0)
    return outx2, outbox

## Crop and Save Train Images

In [15]:

def save_images(images, labels, names, path):
    for im, lab, nam in zip (images, labels, names):
        # Name of the Score
        image_score_class = "Score_"+str(lab+1)
        
        path_ = os.path.join(path,image_score_class,nam)
        
        # Reorder Image axis
        reorder_im = np.moveaxis(im, 0, -1)
        
        # Extend the values to 0-255
        reorder_im = reorder_im*255
        
        # Save Image
        cv2.imwrite(path_, reorder_im)

In [16]:

model_bbox.eval()
for bi, data in tqdm(enumerate(train_loader), total=len(train_loader)):
    images = data[0].to(device)
    labels = data[1].numpy()
    names = data[2]
    
    # Get cropped images
    new_images, _ = get_cropped_img_batch_base_on_CAM(images, device, cam, threshold = BBOX_THRESHOLD)#.to(device)
    
    # Inverse the normalization, and change it to Numpy array
    new_images_arr = invTrans(new_images).detach().cpu().numpy()
    
    # save the images
    save_images(new_images_arr, labels, names, save_cropedImg_path+"Train")

100%|██████████| 144/144 [06:33<00:00,  2.73s/it] 


## Crop and Save Test Images

In [17]:

def save_images_test(images, names, path):
    for im, nam in zip (images, names):
        
        path_ = os.path.join(path,nam)
        
        # Reorder Image axis
        reorder_im = np.moveaxis(im, 0, -1)
        
        # Extend the values to 0-255
        reorder_im = reorder_im*255
        
        # Save Image
        cv2.imwrite(path_, reorder_im)

In [18]:
model_bbox.eval()
for bi, data in tqdm(enumerate(test_loader), total=len(test_loader)):
    images = data[0].to(device)
    names = data[1]
    
    # Get cropped images
    new_images, _ = get_cropped_img_batch_base_on_CAM(images, device, cam, threshold = BBOX_THRESHOLD)#.to(device)
    
    # Inverse the normalization, and change it to Numpy array
    new_images_arr = invTrans(new_images).detach().cpu().numpy()
    
    # save the images
    save_images_test(new_images_arr, names, save_cropedImg_path+"Test")

100%|██████████| 48/48 [00:45<00:00,  1.06it/s]
